In [ ]:
import os
import pandas as pd
import time
from azure.storage.blob import BlobServiceClient

# Azure config
storage_account_url = "https://rvnstorage123.blob.core.windows.net/"
container_name = "raw"
folder_name = "stream_input"
sas_token = "?sp=racw&st=2025-06-23T11:20:38Z&se=2025-06-23T19:20:38Z&spr=https&sv=2024-11-04&sr=c&sig=%2BavHvxnH%2Br83L6qNj4NkBoV%2BR63rYrFNTOX1xaHpO2Y%3D" 

# Init blob client
blob_service_client = BlobServiceClient(account_url=storage_account_url, credential=sas_token)
container_client = blob_service_client.get_container_client(container_name)

def generate_csv(index):
    data = {
        "id": [index],
        "name": [f"Name{index}"],
        "salary": [1000 + index * 100]
    }
    df = pd.DataFrame(data)
    filename = f"employee_{index}.csv"
    df.to_csv(filename, index=False)
    return filename

def upload_to_adls(file_name):
    blob_path = f"{folder_name}/{file_name}"
    with open(file_name, "rb") as data:
        container_client.upload_blob(name=blob_path, data=data)
    print(f"✅ Uploaded {file_name} to {blob_path}")
    os.remove(file_name)

# Simulate streaming data
for i in range(1, 10):
    file_name = generate_csv(i)
    upload_to_adls(file_name)
    time.sleep(60) 
